In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

from google.colab import files

import magenta
import note_seq

print(magenta.__version__)

Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 5.6 MB 4.7 MB/s 
     |████████████████████████████████| 51 kB 5.3 MB/s 
     |████████████████████████████████| 1.4 MB 5.0 MB/s 
     |████████████████████████████████| 2.3 MB 17.8 MB/s 
     |████████████████████████████████| 1.6 MB 22.3 MB/s 
     |████████████████████████████████| 352 kB 53.

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


2.1.3


# Placing notes manually

In [ ]:
from note_seq.protobuf import music_pb2

seq = music_pb2.NoteSequence()

seq.notes.add(pitch=60, start_time=0.0, end_time=0.5, velocity=80)
seq.notes.add(pitch=63, start_time=0.0, end_time=0.5, velocity=80)
seq.notes.add(pitch=60, start_time=0.5, end_time=1.0, velocity=80)
seq.notes.add(pitch=67, start_time=1.0, end_time=1.5, velocity=80)
seq.notes.add(pitch=67, start_time=1.5, end_time=2.0, velocity=80)
seq.notes.add(pitch=69, start_time=2.0, end_time=2.5, velocity=80)
seq.notes.add(pitch=69, start_time=2.5, end_time=3.0, velocity=80)
seq.notes.add(pitch=67, start_time=3.0, end_time=4.0, velocity=80)

seq.total_time = 4

seq.tempos.add(qpm=60);

# plot
note_seq.plot_sequence(seq)

# audio player
note_seq.play_sequence(seq, synth=note_seq.fluidsynth)

# Generating music via rules

In [ ]:
import numpy as np
from note_seq.protobuf import music_pb2

class Cell:
  def __init__(self, pitch):
    self.pitch = pitch
    self.duration = 0.5
    self.velocity = 80
    #print(f"Created Cell with pitch {pitch}")

In [ ]:
# Blank base rule
class Rule:
  def __init__(self):
    pass

  def evaluate(self, notes, current_index):
    return Cell(60)

In [ ]:
# Increasing pitch rule
class IncreaseRule(Rule):
  def __init__(self, base_pitch):
    self.base_pitch = base_pitch

  def evaluate(self, notes, current_index):
    if current_index == 0 or len(notes) == 0:
      return Cell(self.base_pitch)
    # np.nonzero(notes[current_index-1,:])[0]
    return Cell(notes[current_index-1].pitch + 1) 

In [ ]:
class Board:
  def __init__(self):
    self.notes = []
    self.seq = seq = music_pb2.NoteSequence()

  def generate_cells(self, length, rule):
    for i in range(length):
      note = rule.evaluate(self.notes, i)
      self.notes.append(note)

  def generate_cells_sequence(self, length, rule):
    for i in range(length):
      column_notes = rule.evaluate(self.notes, i)
      self.notes.append(column_notes)

  def generate_sequence(self):
    time = 0.0
    for i,n in enumerate(self.notes):     
        self.seq.notes.add(pitch=n.pitch, start_time=time, end_time=time+n.duration, velocity=n.velocity)
        time += n.duration

    self.seq.total_time = time
    self.seq.tempos.add(qpm=60);

  def generate_sequence_next(self):
    time = 0.0
    for i in self.notes:
      for n in i:       
        self.seq.notes.add(pitch=n.pitch, start_time=time, end_time=time+n.duration, velocity=n.velocity)
      time += n.duration

    self.seq.total_time = time
    self.seq.tempos.add(qpm=60);

  def plot(self):
    note_seq.plot_sequence(self.seq)

  def play(self):
    note_seq.play_sequence(self.seq, synth=note_seq.fluidsynth)

In [ ]:
rule = Rule()

b = Board()
b.generate_cells(10, rule)
b.generate_sequence()
b.plot()
b.play()

In [ ]:
rule = IncreaseRule(base_pitch=38)

b = Board()
b.generate_cells(4, rule)
b.generate_sequence()
b.plot()
b.play()

In [ ]:
# Elementary Cellular Automaton
class ElemCARule(Rule):
  def __init__(self, base_pitch, width, rule):
    self.base_pitch = base_pitch
    self.last_row = np.zeros((width), dtype = int)
    self.width = width
    self.notes = []
    self.rule = rule

  def nextCell(prev, mid, nxt, bin_rule):
    # TODO Your implementation
    # Implement rule here
    
    if(prev == mid == nxt ==1):
        mid =  int(bin_rule[0])
    elif prev == mid ==1 and nxt ==0:
        mid =  int(bin_rule[1])
    elif prev == nxt ==1 and mid==0:
        mid =  int(bin_rule[2])
    elif prev==1 and mid == nxt==0:
        mid =  int(bin_rule[3])
    elif prev==0 and mid == nxt==1:
        mid =  int(bin_rule[4])
    elif prev == nxt ==0 and mid ==1:
        mid =  int(bin_rule[5])
    elif prev == mid ==0 and nxt==1:
        mid =  int(bin_rule[6])
    elif prev == mid == nxt==0:
        mid = int(bin_rule[7])
    
    return mid

  def evaluate(self, notes, current_index):
    self.notes = []
    # First row always has one black cell in the middle 
    if current_index == 0 or len(notes) == 0:
      self.last_row[int(self.width/2)] = 1
      self.notes.append(Cell(self.base_pitch))
      return self.notes

    bin_rule = "{0:{fill}8b}".format(self.rule, fill='0')
    # First row always has one black cell in the middle 
    
    for j in range(0,self.width):
        nextRow = np.zeros(self.width, dtype = int)
        #Boundary conditions in range
        for i in range(1, self.width-1):
            nextRow[i] = ElemCARule.nextCell(self.last_row[i-1], self.last_row[i], self.last_row[i+1], bin_rule)
    self.last_row = nextRow
    
    for i in range(0,self.width):
      if self.last_row[i]==1:
        self.notes.append(Cell(int(self.base_pitch-self.width/2+2*i)))
    return self.notes 

In [ ]:
rule = ElemCARule(base_pitch=60,width=20,rule=52)

b = Board()
b.generate_cells_sequence(10, rule)
b.generate_sequence_next()
b.plot()
b.play()



In [ ]:
rule = ElemCARule(base_pitch=60,width=11,rule=145)

b = Board()
b.generate_cells_sequence(40, rule)
b.generate_sequence_next()
b.plot()
b.play()

In [ ]:
rule = ElemCARule(base_pitch=55,width=5,rule=163)

b = Board()
b.generate_cells_sequence(40, rule)
b.generate_sequence_next()
b.plot()
b.play()